<a href="https://colab.research.google.com/github/emamanni/AnalisiDeiDati24-25/blob/main/11_LibreriaPuLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Libreria PuLP**

PuLP è un modulo Python che consente agli utenti di descrivere e risolvere problemi di ottimizzazione. PuLP lavora interamente all'interno della sintassi e dei naturali idiomi di Python fornendo oggetti Python che rappresentano problemi di ottimizzazione e variabili di decisione e permettendo ai vincoli di essere espressi in un modo che è molto simile all'originale matematico.

Il primo passo è installare la libreria, se non già fatto in precendenza

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 1.7 MB/s eta 0:00:00


Supponiamo di voler utilizzare PuLP per risolvere il seguente problema di mix di produzione:

$\mathrm{Max}\, z = 15x_A + 10x_B$ \\
s.v. \\
$4x_A + 2x_B \leq 400$ \\
$2x_A + 4x_B \leq 400$ \\
$x_A \leq 40$ \\
$x_B \leq 120$ \\
$x_A, x_B \geq 0$

Innanzitutto, occorre importare la libreria.

In [ ]:
from pulp import *

## **Modello in forma estesa**
Definiamo le diverse componenti del modello. Si crea una variabile chiamata `prob` (ovviamente, il nome può essere modificato), utilizzando la funzione `LpProblem`. Essa accetta due parametri: il primo è un nome arbitrario da assegnare al problema (una stringa); il secondo può essere `LpMinimize` oppure `LpMaximize`, a seconda del tipo di probema da risolvere.

In [ ]:
# Creiamo la variabile 'prob' per contenere i dati del problema
prob = LpProblem("ProblemaMixProduzione", LpMaximize)

Le variabili del problema $x_A$ e $x_B$ sono create utilizzando la classe `LpVariable`. Essa ha quattro parametri: il primo è un nome che può essere scelto in maniera arbitraria, il secondo è il limite inferiore di questa variabile, il terzo è il limite superiore e il quarto è essenzialmente il tipo di valori che la variabile può assumere (discreti o continui). Le opzioni per il quarto parametro sono `LpContinuous` (predefinito) o `LpInteger`. I limiti possono essere inseriti direttamente come valore numerico oppure `None` per non impostare alcun limite (cioè, più o meno infinito, con quest'ultimo che è il valore predefinito). Per il problema di mix di produzione:

In [ ]:
# Creiamo le due variabili del problema, di tipo intero, con limite inferiore pari a 0
xA = LpVariable("x_A", 0, None, LpInteger)
xB = LpVariable("x_B", 0, None, LpInteger)

A questo punto, si aggiungono dati alla variabile `prob`, utilizzando l'operatore `+=`. In primo luogo occorre aggiungere la funzione obiettivo, seguita da una virgola e da una stringa di testo descrittiva.

In [ ]:
# Aggiungiamo la funzione obiettivo a 'prob'
prob += 15 * xA + 10 * xB, "Profitto complessivo"

Successivamente, si aggiungono i vincoli del problema, sempre utilizzando l'operatore `+=`. Come nel caso della funzione obiettivo, dopo l'espressione relativa al vincolo occorre inserire una virgola e una stringa descrittiva. Eventuali vincoli di non-negatività sono già stati inseriti in fase di definizione delle variabili.

In [ ]:
# Aggiungiamo esplicitamente i 4 vincoli funzionali
prob += 4 * xA + 2 * xB <= 400, "Utilizzo reparto 1"
prob += 2 * xA + 4 * xB <= 400, "Utilizzo reparto 2"
prob += xA <= 40, "Domanda del prodotto A"
prob += xB <= 120, "Domanda del prodotto B"

A questo punto possiamo risolvere il problema utilizzando la funzione `solve()`, specificando o meno il nome del solver da utilizzare (ad esempio, `prob.solve(CPLEX())` per utilizzare il solver IBM CPLEX).

In [ ]:
# Risolviamo il problema utilizzando il solver di default
prob.solve()

1

A valle della risoluzione, recuperiamo lo status della soluzione, i cui possibili valori sono:  “Not Solved”, “Infeasible”, “Unbounded”, “Undefined” oppure “Optimal”. Successivamente, se il problema è stato risolto all'ottimo, recuperiamo il valore ottimale delle variabili. Tali valori sono stampati a schermo insieme al valore ottimo della funzione obiettivo.

In [ ]:
# Stampiamo a schermo lo status della soluzione
print("Status:", LpStatus[prob.status])

if prob.status == LpStatusOptimal:
  # Stampiamo il valore di ciascuna variabile nella soluzione ottima
  for v in prob.variables():
    print(v.name, "=", v.varValue)

# Stampiamo il valore ottimo di funzione obiettivo
print("Profitto complessivo =", value(prob.objective))

Status: Optimal
x_A = 40.0
x_B = 80.0
Profitto complessivo = 1400.0


## **Modello in forma compatta**
È possibile anche definire e risolvere un modello matematico espresso in forma compatta, ovvero in una forma del tipo:

$\mathrm{Max}\, z = \mathbf{c}^T \mathbf{x}$ \\
s.v. \\
$A \mathbf{x} \leq \mathbf{b}$ \\
$\mathbf{x} \geq \mathbf{0}$

Come nel caso precedente, occorre importare la libreria e definire una variabile che conterrà i dati del problema.

In [ ]:
from pulp import *

# Creiamo la variabile 'prob' per contenere i dati del problema
prob = LpProblem("ProblemaMixProduzione", LpMaximize)

Prima di definire la funzione obiettivo, le variabili ed i vincoli da aggiungere a `prob`, occorre definire le strutture dati contenenti i parametri del problema da utilizzarsi nella funzione obiettivo e nei vincoli. A tal proposito, creiamo dapprima una lista relativa ai prodotti da realizzare ed una relativa ai reparti. Successivamente, definiamo le strutture dati (in questo caso dei dizionari) relative ai profitti, alla domanda dei prodotti, alle ore di lavorazione dei prodotti nei diversi reparti e la capacità dei reparti stessi.

In [ ]:
# lista dei prodotti
prodotti = ['A', 'B']

# lista dei reparti
reparti = ['reparto1', 'reparto2']

# dizionario relativo alle ore di lavorazione necessarie
ore = {
    'A': {'reparto1': 4, 'reparto2': 2},
    'B': {'reparto1': 2, 'reparto2': 4}
}

# dizionario relativo alla capacità dei diversi reparti
capacita = {'reparto1': 10, 'reparto2': 10}

# ore di lavorazione disponibili settimanalmente
ore_per_settimana = 40

# dizionario relativo alla domanda dei diversi prodotti
domanda = {'A': 40, 'B':120}

# dizionario relativo al profitto unitario dei prodotti
profitto = {'A': 15, 'B':10}

Il prossimo passo è definire un dizionario contenente le variabili, avente come chiavi i nomi dei prodotti. Successivamente aggiungiamo la funzione obiettivo ed i vincoli funzionali del problema.

In [ ]:
# Creiamo un dizionario contenente  le variabili
x = LpVariable.dicts('x', prodotti, 0, None, LpInteger)

# Aggiungiamo la funzione obiettivo
prob += lpSum(profitto[i] * x[i] for i in prodotti)

for j in reparti:
  prob += lpSum(ore[i][j] * x[i] for i in prodotti) <= ore_per_settimana * capacita[j]

for i in prodotti:
  prob += x[i] <= domanda[i]

Infine, risolviamo il problema e stampiamo la soluzione.

In [ ]:
# Risolviamo il problema utilizzando il solver di default
prob.solve()

# Stampiamo a schermo lo status della soluzione
print("Status:", LpStatus[prob.status])

if prob.status == LpStatusOptimal:
  # Stampiamo il valore di ciascuna variabile nella soluzione ottima
  for i in prodotti:
    if x[i].varValue > 0:
      print(x[i].name, '=', x[i].varValue)

# Stampiamo il valore ottimo di funzione obiettivo
print("Profitto complessivo =", value(prob.objective))

Status: Optimal
x_A = 40.0
x_B = 80.0
Profitto complessivo = 1400.0
